In [1]:
import logging
import wandb
import torch
import os
import json

from pathlib import Path
from lightning.pytorch import (
    callbacks,
    loggers,
    Trainer,
    utilities
)

from model import Digits
from data_module import MNISTDataModule
from extentions.ext.callbacks import DiffEarlyStopping, EarlyStopping

ModuleNotFoundError: No module named 'extentions'

In [4]:
root_path = Path('../')

dm = MNISTDataModule(
    data_dir=(root_path / 'data').as_posix(),
)

In [10]:
logging.getLogger("lightning.pytorch").setLevel(logging.INFO)

# Training

In [12]:
model = Digits(
    optimizer_name='SGD',
    optimizer_hparams={
        'lr': 0.001,
        'momentum': 0.9
    }
)

earlystopping_callbacks = [
    DiffEarlyStopping(
        monitor1="val_loss",
        monitor2="train_loss",
        diff_threshold=0.05, # like val_loss=0.09, train_loss=0.04
        patience=5,
        verbose=True
    ),
    # EarlyStopping(
    #     monitor="val_acc",
    #     min_delta=0.0,
    #     mode='max',
    #     stopping_threshold=99.99,
    #     patience=5,
    #     verbose=True
    # ),
]

checkpoint_callback = callbacks.ModelCheckpoint(
    filename="epoch={epoch}-loss={val_loss:.3f}",
    auto_insert_metric_name=False,
    monitor='val_loss',
    mode='min',
    save_top_k=3
)

In [13]:
utilities.model_summary.ModelSummary(model)

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 322 K 
-------------------------------------
322 K     Trainable params
0         Non-trainable params
322 K     Total params
1.288     Total estimated model params size (MB)

In [14]:
# Number of training batches
print(torch.ceil(torch.tensor(len(dm.train_dataset) / 64)))
print(torch.ceil(torch.tensor(len(dm.valid_dataset) / 64)))

tensor(657.)
tensor(282.)


In [15]:
max_time =  {'minutes': 20} if torch.cuda.is_available() else {'hours': 2}

log_dir = root_path/'logs'
log_dir.mkdir(exist_ok=True)

api_key = None
try:
    with open(root_path/'secrets.json') as f:
        secrets = json.load(f)
    api_key = secrets.get("WANDB_API_KEY")
except FileNotFoundError:
    pass

logger = loggers.WandbLogger(
    project='digits',
    save_dir=log_dir,
    log_model='all',
    # api_key=api_key
)

trainer = Trainer(
    min_epochs=10,
    max_epochs=50,
    log_every_n_steps=1,
    max_time=max_time,
    logger=logger,
    callbacks=[checkpoint_callback] + earlystopping_callbacks,
    enable_model_summary=False,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(model, train_dataloader, valid_dataloader)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

c:\Dev\digits\.venv\lib\site-packages\lightning\pytorch\trainer\call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
wandb.finish()

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████
train_acc_epoch,▁
train_acc_step,▅▁▂▂▅▅▃▅█▄▂▆▆▇▆▇█▄▂▆▅▃
train_loss_epoch,▁
train_loss_step,▃█▇▅▇▅▆▃▁▆▅▆▅▄▂▄▂▃█▂▄▆
trainer/global_step,▂▄▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▆█
val_acc_epoch,▁
val_acc_step,▃▅▃▃▅▂▆▄▃▄▅▁▃▂▁▂▄▅▂▃█▄▂▆▁▃▂▄▂▅▅▃▅▃▅▄▅▄▂▂
val_loss_epoch,▁
val_loss_step,▅▃▇▅▅▆▁▄▄▇▃█▅▇▆▆▅▅▇▆▃▆▇▂█▆▆▁▆▄▆▅▄▆▄▆▅▇▆▆
epoch,1


In [ ]:
checkpoint_callback.best_model_path

'..\\logs\\digits\\z9rm2ase\\checkpoints\\epoch=0-loss=2.301.ckpt'